### Imports & Querie Loading

In [ ]:
import sys
import os

#get the reddit web scraper library
folder_path = os.path.abspath("..")

if folder_path not in sys.path:
    sys.path.append(folder_path)

from database import KnowledgeBase
import reddit

In [ ]:
import json
#get queries text example
with open("../gradio-tests/queries_0a891.json") as file:
    out_json = json.load(file)

### Initial Web Search

In [ ]:
from googlesearch import search
from datetime import datetime

#build database
db = KnowledgeBase()
#start session
db.start_session()

#save topic
topic = "javascript"
#get difficulty
difficulty = "beginner"
#get search query
search_query = out_json[difficulty]["query"]
#create search
search_results = search(search_query, advanced=True, num_results=5)

index = 0
#search results
results_data = []
#loop through results
for result in search_results:
    index += 1

    print(f"Round {index}")
    if not db.find_url(result.url):
        #track current result
        curr_data = {}
        #save resource url
        curr_data["resource"] = result.url
        #save resource title
        curr_data["title"] = result.title
        #save resource description
        curr_data["description"] = result.description.replace("'", "")
        #save resource topic
        curr_data["topic"] = topic
        #save resource difficulty
        curr_data["difficulty"] = difficulty
        #save resource validation
        curr_data["validated"] = False
        #save resource found time
        curr_data["found_time"] = datetime.now()
        #append data to list
        results_data.append(curr_data)
    else:
        print(f"Link: {result.url} already exists in database")
        continue

    #check to see if we found our five links
    if len(results_data) == 5:
        break

#check for missing resources
if len(results_data) < 5:
    #build search query for reddit
    reddit_query = f"Reddit {out_json[difficulty]["query"]}"
    #get reddit search results
    reddit_results = search(reddit_query, advanced=True, num_results=5)

    index = 0
    #go through search results
    for result in reddit_results:
        
        index += 1
        print(f"Reddit Round: {index}")

        if result.url.split(".")[1] == "reddit":
            #get the scraped resources
            scraped_resources = reddit.get_links(result.url)
            #check to see if we get scraped resources
            if scraped_resources:
                #loop through scraped resources
                for scraped_url in scraped_resources:
                    #reverse search url for detailed information
                    web_results = search(scraped_url, advanced=True, num_results=5)

                    #check to see if link is already in the database
                    for scraped_result in web_results:
                        #make sure current link is not already in the database
                        if not db.find_url(scraped_result.url):
                            #track current result
                            curr_data = {}
                            #save resource url
                            curr_data["resource"] = scraped_result.url
                            #save resource title
                            curr_data["title"] = scraped_result.title
                            #save resource description
                            curr_data["description"] = scraped_result.description.replace("'", "")
                            #save resource topic
                            curr_data["topic"] = topic
                            #save resource difficulty
                            curr_data["difficulty"] = difficulty
                            #save resource validation
                            curr_data["validated"] = False
                            #save resource found time
                            curr_data["found_time"] = datetime.now()
                            #break out of current loop
                            if curr_data:
                                #append data to list
                                results_data.append(curr_data)
                                break
                        else:
                            #break out of loop since url is already in database
                            print(f"Link: {scraped_result.url} already exists in database")
                            break
                    if len(results_data) >= 5:
                        break
                if len(results_data) >= 5:
                    break

#check to see if we have enough data to push to database
if len(results_data) >= 5:
    #loop through search results and insert
    for data in results_data:
        #insert current data into database
        db.insert_resource(data)
else:
    print("Not enough data to commit")

#commit changes made in session
db.commit_session()
#end session
db.end_session()